Task-I: Build and populate necessary tables (30% of course project
grade)
• Ingest the data from all years (Male: 2015-2022 and Female: 2016-2022) into
one Postgres Database table.
o Conduct any column name changes to ensure data from various years are
properly aligned in the correct columns in your DB table.
• Add a new column for the year. Also, ensure every record can be uniquely
identified in the database table.
• Your tables should be created in schema with the name “fifa”.
• In your ReadMe.md, add a description of the features in the dataset.
• In your ReadMe.md file, comment on the benefit of using PostgreSQL DB table
compared to a NoSQL Database in this case.

In [15]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType  
appName = "FifaProject"
master = "local" 
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.jars", "/Users/kozasound/Desktop/14_763_AISTC/postgresql-42.6.2.jar") \
    .getOrCreate()
print("Spark session started successfully with PostgreSQL driver")

Spark session started successfully with PostgreSQL driver


In [13]:
from pyspark.sql.functions import lit, when, col
from pyspark.sql.types import BooleanType, DoubleType, IntegerType

db_properties = {
    'user': 'fifaproject',
    'password': '18763', 
    'url': 'jdbc:postgresql://localhost:5432/fifaproject',
    'dbtable': 'fifa.players_data',
    'driver': 'org.postgresql.Driver'
}

male_files = [f'/Users/kozasound/Desktop/14_763_AISTC/shawnakk_Project_Option1/archive/players_{year}.csv' for year in range(15, 23)]
male_years = [2000 + year for year in range(15, 23)]
female_files = [f'/Users/kozasound/Desktop/14_763_AISTC/shawnakk_Project_Option1/archive/female_players_{year}.csv' for year in range(16, 23)]
female_years = [2000 + year for year in range(16, 23)]


files = female_files + male_files
years = female_years + male_years
genders = ['Female'] * len(female_files) + ['Male'] * len(male_files)

for file, year, gender in zip(files, years, genders):
    try:
        df = spark.read.csv(file, header=True, inferSchema=True)
        df = df.withColumn("year", lit(year)).withColumn("sex", lit(gender))
        df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])
        columns_to_cast = {
            'value_eur': DoubleType(),
            'wage_eur': DoubleType(),
            'club_team_id': IntegerType(),
            'league_level': IntegerType(),
            'club_jersey_number': IntegerType()
        }
        for column, col_type in columns_to_cast.items():
            if column in df.columns:
                df = df.withColumn(column, col(column).cast(col_type))
        if 'real_face' in df.columns:
            df = df.withColumn("real_face", when(col("real_face") == "Yes", True).when(col("real_face") == "No", False).otherwise(None).cast(BooleanType()))
        df.write.format("jdbc") \
            .mode("append") \
            .option("url", db_properties['url']) \
            .option("dbtable", db_properties['dbtable']) \
            .option("user", db_properties['user']) \
            .option("password", db_properties['password']) \
            .option("driver", db_properties['driver']) \
            .save()
        print(f"Data for year {year}, gender: {gender} written to PostgreSQL.")
    except Exception as e:
        print(f"An error occurred while processing year {year}, gender: {gender}: {e}")

try:
    sample_df = spark.read \
        .format("jdbc") \
        .option("url", db_properties['url']) \
        .option("dbtable", "(SELECT * FROM fifa.players_data LIMIT 5) AS sample") \
        .option("user", db_properties['user']) \
        .option("password", db_properties['password']) \
        .option("driver", db_properties['driver']) \
        .load()
    sample_df.show()
except Exception as e:
    print(f"An error occurred while reading sample data: {e}")

spark.stop()


Data for year 2016, gender: Female written to PostgreSQL.
Data for year 2017, gender: Female written to PostgreSQL.
Data for year 2018, gender: Female written to PostgreSQL.
Data for year 2019, gender: Female written to PostgreSQL.
Data for year 2020, gender: Female written to PostgreSQL.
Data for year 2021, gender: Female written to PostgreSQL.
Data for year 2022, gender: Female written to PostgreSQL.


Data for year 2015, gender: Male written to PostgreSQL.


Data for year 2016, gender: Male written to PostgreSQL.


Data for year 2017, gender: Male written to PostgreSQL.


Data for year 2018, gender: Male written to PostgreSQL.


Data for year 2019, gender: Male written to PostgreSQL.


Data for year 2020, gender: Male written to PostgreSQL.


Data for year 2021, gender: Male written to PostgreSQL.


Data for year 2022, gender: Male written to PostgreSQL.
+---------+--------------------+----------+------------------+----------------+-------+---------+---------+--------+---+----------+---------+---------+------------+---------+-----------+------------+-------------+------------------+----------------+-----------+-------------------------+--------------+----------------+--------------+---------------+--------------------+--------------+---------+-----------+------------------------+-------------+----------------+---------+------------------+--------------------+--------------------+----+--------+-------+---------+---------+------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+-----------

Task-II: Conduct analytics on your dataset (20% of course project
grade)
Develop Python functions that run Spark to answer the following questions (given that x,
y and z) are user-entered parameters. Core analysis should be conducted via Spark
and data should be ingested from Postgres database.
• In Year X, what were the Y clubs that had the highest number of players with
contracts ending in year Z (or after)?
o X is a year between (2015 and 2022, inclusively).
o Y is a positive integer.
o Z is a year that can hold the value of 2023 or a year after it.
• In sports, maturity and energy of teams depend on the average age of team
players (among other factors). Therefore, it’s important to have a function that
can find clubs with such features.
o List the X clubs with the highest (or lowest) average player age for a given
year Y.
▪ X represents a positive integer, but you should handle a scenario if
X is not positive value.
▪ Y represents a year between 2015 and 2022 inclusively.
▪ Provide the user with the ability to choose if they want the highest
average age or the lowest average age.
▪ Make sure to handle this scenario as well: if the user requests 5
clubs with highest averages but there are 3 clubs that share the
same count at rank number 5, please include all of them in your
output
• What is the most popular nationality in the dataset for each year? (i.e. display the
most frequent nation for 2015, 2016, etc.).

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, max as max_, avg

print("Bullet Point 1")
def top_clubs_by_contracts(spark, x_year, y_clubs, z_contract):
    df = spark.read \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://localhost:5432/fifaproject") \
        .option("dbtable", "fifa.players_data") \
        .option("user", "fifaproject") \
        .option("password", "18763") \
        .option("driver", "org.postgresql.Driver") \
        .load()
    
    filt_df = df.filter((col("year") == x_year) & (col("club_contract_valid_until") >= z_contract) & (col("sex") == "Male"))
    club_counts = filt_df.groupBy("club_name") \
                         .agg(count("sofifa_id").alias("player_count")) \
                         .orderBy(col("player_count").desc()) \
                         .limit(y_clubs)
    club_counts.show()

top_clubs_by_contracts(spark, 2020, 5, 2024)

print("Bullet Point 2")
def clubs_by_avg_age(spark, y_year, x_clubs, order='highest'):
    df = spark.read.format("jdbc") \
        .option("url", db_properties['url']) \
        .option("dbtable", db_properties['dbtable']) \
        .option("user", db_properties['user']) \
        .option("password", db_properties['password']) \
        .option("driver", db_properties['driver']) \
        .load()
    
    filt_df = df.filter((col("year") == y_year) & (col("sex") == "Male"))
    avg_age_df = filt_df.groupBy("club_name").agg(avg("age").alias("avg_age"))
    sorted_df = avg_age_df.orderBy(col("avg_age").desc() if order == 'highest' else col("avg_age").asc())
    top_df = sorted_df.limit(x_clubs)
    max_age = top_df.select("avg_age").collect()[-1][0] if top_df.count() == x_clubs else None
    res_df = sorted_df.filter(col("avg_age") >= max_age) if max_age else top_df
    res_df.show()

clubs_by_avg_age(spark, 2020, 5, order='highest')

print("Bullet Point 3")
def most_popular_nationality(spark):
    df = spark.read.format("jdbc") \
        .option("url", db_properties['url']) \
        .option("dbtable", db_properties['dbtable']) \
        .option("user", db_properties['user']) \
        .option("password", db_properties['password']) \
        .option("driver", db_properties['driver']) \
        .load()
    
    nat_counts = df.filter(col("sex") == "Male").groupBy("year", "nationality_name").agg(count("*").alias("count"))
    max_per_year = nat_counts.groupBy("year").agg(max_("count").alias("max_count"))
    most_popular = nat_counts.alias("nc").join(
        max_per_year.alias("mpy"),
        (col("nc.year") == col("mpy.year")) & (col("nc.count") == col("mpy.max_count"))
    ).select(col("nc.year"), col("nc.nationality_name"), col("nc.count")).orderBy("nc.year")
    most_popular.show()

most_popular_nationality(spark)


Bullet Point 1
+-------------------+------------+
|          club_name|player_count|
+-------------------+------------+
|   Deportes Iquique|          12|
|Patriotas Boyacá FC|          12|
|          Al Ain FC|          11|
|  Alianza Petrolera|          11|
|     Atlético Huila|          11|
+-------------------+------------+

Bullet Point 2
+--------------------+-------+
|           club_name|avg_age|
+--------------------+-------+
|           Fortaleza|   32.6|
|            Cruzeiro|   31.6|
|Club Athletico Pa...|   31.4|
|            Botafogo|   31.4|
|Associação Chapec...|   31.4|
+--------------------+-------+

Bullet Point 3
+----+----------------+-----+
|year|nationality_name|count|
+----+----------------+-----+
|2015|         England| 1627|
|2016|         England| 1519|
|2017|         England| 1627|
|2018|         England| 1633|
|2019|         England| 1625|
|2020|         England| 1670|
|2021|         England| 1685|
|2022|         England| 1719|
+----+----------------+-----+